# PCR-GLOBWB 2

In [1]:
from os import environ, remove
from configparser import ConfigParser
from glob import glob
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from cartopy import crs
from cftime import num2date

import hydrostats.metrics as hm

import pickle

from grpc4bmi.bmi_client_docker import BmiClientDocker
from ewatercycle.observation.grdc import get_grdc_data

In [2]:
def var_to_xarray(model, variable):
    # Get grid properties from model (x = latitude !!)
    # could be speedup, lots of bmi calls are done here that dont change between updates
    shape = model.get_grid_shape(model.get_var_grid(variable))
    lat = model.get_grid_x(model.get_var_grid(variable))
    lon = model.get_grid_y(model.get_var_grid(variable))
    time = num2date(model.get_current_time(), model.get_time_units())

    # Get model data for variable at current timestep
    data = model.get_value(variable)
    data = np.reshape(data, shape)

    # Create xarray object
    da = xr.DataArray(data, 
                      coords = {'longitude': lon, 'latitude': lat, 'time': time}, 
                      dims = ['latitude', 'longitude'],
                      name = variable,
                      attrs = {'units': model.get_var_units(variable)}
                     )

    # Masked invalid values on return array:
    return da.where(da != -999)

In [3]:
# currently global data are available on cartesius
input_dir='/mnt/data/examples/technical_paper/pcr-globwb/input/'
output_dir='/mnt/home/user43/Sophie_Bep/output_Merrimack'
forcing_dir = '/mnt/data/examples/technical_paper/pcr-globwb/input/'

setting_file = '/mnt/home/user43/Sophie_Bep/Merrimack/Sophie_merrimack_05min_era5.ini'

In [4]:
model = BmiClientDocker(image='ewatercycle/pcrg-grpc4bmi:latest', image_port=55555,

                   input_dir=input_dir,

                   output_dir=output_dir)

 

model.initialize(setting_file)

In [5]:
output = []
time_units = model.get_time_units()
print(f'Running pcrglobwb', flush=True)
while model.get_current_time() < model.get_end_time():
    time = num2date(model.get_current_time(), time_units)
    print(f'Current time: {time}', end="\r")
    model.update()
    output.append(var_to_xarray(model, variable="total_evaporation"))
print('')

Running pcrglobwb
Current time: 2008-12-30 00:00:00


In [6]:
data  = xr.concat(output, dim='time')
data

<xarray.DataArray 'total_evaporation' (time: 3653, latitude: 54, longitude: 54)>
array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.39119967e-04, 2.39013636e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.51863147e-04, 1.54188907e-04, 1.49376778e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.05564322e-04, 2.27758806e-04, 2.48860772e-04, ...,
         8.20149507e-05, 9.40026366e-05, 9.76992960e-05],
        [1.33953508e-04, 1.98692811e-04, 2.30703328e-04, ...,
         1.00796671e-04, 7.30685861e-05, 7.20308817e-05],
        [1.26329949e-04, 2.15336084e-04, 1.95385364e-04, ...,
         7.11917528e-05, 6.89976732e-05, 6.52228555e-05]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.39399204e-04, 2.39281915e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.52048189e-04, 1.54379188e-04, 1.49560801e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.05731073e-04, 2.28158751e-04, 2.49252800e-04, ...,
         8.24151139e-05, 9.44635976e-05, 9.81732810e-05],
        [1.34166068e-04, 1.99008209e-04, 2.31068581e-04, ...,
         1.01165366e-04, 7.34237037e-05, 7.23780249e-05],
        [1.26531391e-04, 2.15675158e-04, 1.95696965e-04, ...,
         7.15318456e-05, 6.93214679e-05, 6.55120166e-05]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.39704794e-04, 2.39568850e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.52250752e-04, 1.54587571e-04, 1.49762054e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.05913467e-04, 2.28593082e-04, 2.49681587e-04, ...,
         8.28276970e-05, 9.49392561e-05, 9.86626328e-05],
        [1.34398506e-04, 1.99353075e-04, 2.31467973e-04, ...,
         1.01549194e-04, 7.37898081e-05, 7.27359948e-05],
        [1.26751707e-04, 2.16045839e-04, 1.96037712e-04, ...,
         7.18826777e-05, 6.96556162e-05, 6.58108984e-05]],

       ...,

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 3.44647298e-04, 3.44634260e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.29987869e-04, 2.35192128e-04, 2.29188445e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.93894084e-04, 4.90918988e-04, 5.31536585e-04, ...,
         2.07196441e-04, 2.33399391e-04, 2.41827220e-04],
        [2.31993195e-04, 4.07887943e-04, 4.88964259e-04, ...,
         2.53648410e-04, 1.85673198e-04, 1.83256911e-04],
        [2.22465373e-04, 4.36062546e-04, 3.93456896e-04, ...,
         1.80911738e-04, 1.75664216e-04, 1.66285012e-04]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 3.17762198e-04, 3.17755359e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.11305553e-04, 2.16060202e-04, 2.10497470e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.46890685e-04, 3.73603194e-04, 4.05640982e-04, ...,
         1.22227459e-04, 1.38426389e-04, 1.43720361e-04],
        [1.76225440e-04, 3.10342555e-04, 3.72908253e-04, ...,
         1.48973122e-04, 1.09291897e-04, 1.07823020e-04],
        [1.68817438e-04, 3.33186879e-04, 2.99439387e-04, ...,
         1.06424413e-04, 1.03259088e-04, 9.76100273e-05]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.45390926e-04, 2.45388306e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.000000

In [46]:
PCRGLOB = data.sel(latitude=42.5378, longitude=-72.1715, method='nearest')*1000
ts = PCRGLOB.to_dataframe()
ts.to_csv(r'/mnt/home/user43/Sophie_Bep/Merrimack/PCRGLOB_evap.csv')
tsdag = ts["2002-01-01": '2008-12-31']

In [35]:
lamda = 2.43 * (10 **6)
rho = 1000

Flux_file = "/mnt/home/user43/Sophie_Bep/Merrimack/FLX_US-Ha1_FLUXNET2015_SUBSET_DD_1991-2012_1-4.csv"
Fluxnetdagall = pd.read_csv(Flux_file, skipinitialspace=True, parse_dates=[0],index_col=0)
Fluxnetdag = Fluxnetdagall[["LE_F_MDS"]] / (lamda * rho) * 86400000

In [47]:
new = Fluxnetdag["2002-01-01": '2008-12-31']
Fluxnet = new["LE_F_MDS"]
tsdag = tsdag.join(Fluxnet)
tsdag.to_csv(r'/mnt/home/user43/Sophie_Bep/Merrimack/evap_dag.csv')

In [50]:
model.finalize()